<p style="font-family: Arial; font-size:1.75em;color:#003399; font-style:bold"><br>
 Segmentacja 
</p><br>
W ostatniej części pokazane zostaną sposoby wyodrębniania obiektów z obrazu oraz określanie ich charakterystyki

In [1]:
import skimage
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, img_as_ubyte, img_as_float, img_as_int

In [2]:
# Wczytanie obrazu zarejestrowanego o 8.01.2019 9:45:01 przez MSI, Sentinel-2, ESA
img = io.imread('S2A_MSIL1C_20190108T095401_N0207_R079_Gozo.tif')
gmg = io.imread('S2A_MSIL1C_20190108T095401_N0207_R079_Gozo.tif', as_grey=True) # wczytanie jako 1 czarno-biały kanał

red = img[:,:,2]
green = img[:,:,1]
blue = img[:,:,0]

C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


In [19]:
from skimage import exposure

def contrast_perc(band, d, u):
    lower = np.percentile(band,d)
    upper = np.percentile(band,u)
    rescaled = exposure.rescale_intensity(band,in_range=(lower, upper))
    
    return(rescaled)

# Dopasuj kontrast liniowo pomiędzy percentylami
pD = 7; pU = 98
perc_red = contrast_perc(red, pD, pU)
perc_green = contrast_perc(green, pD, pU)
perc_blue = contrast_perc(blue, pD, pU)
perc = img_as_float(np.stack([perc_red, perc_green, perc_blue], axis=2))

foto = img_as_ubyte(contrast_perc(gmg, 1, 98))

fig = plt.figure(figsize=(10,10))
ax0 = plt.subplot(121)
ax0.imshow(perc)
ax0.set_title("RGB image")
ax1 = plt.subplot(122)
ax1.imshow(foto,cmap=plt.get_cmap('gray'))
ax1.set_title("Grayscale image")

plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


<i>-------Porównanie metod segmentacji-------</i>

In [20]:
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.segmentation import mark_boundaries

fz = felzenszwalb(perc, scale=100, sigma=0.5, min_size=200)
slic = slic(perc, n_segments=250, compactness=10, sigma=1)
quick = quickshift(perc, kernel_size=9, max_dist=6, ratio=0.2)

fig = plt.figure(figsize=(30,30))
ax0 = plt.subplot(311)
ax0.imshow(mark_boundaries(perc, fz))
ax0.set_title("Felzenszwalb segmentation")
ax1 = plt.subplot(312)
ax1.imshow(mark_boundaries(perc, slic))
ax1.set_title("SLIC segmentation")
ax2 = plt.subplot(313)
ax2.imshow(mark_boundaries(perc, quick))
ax2.set_title("Quickshift segmentation")

plt.show()

<i>-------Porównanie wyników segmentacji w zależności od przygotowania warstw wejściowych-------</i>

In [5]:
from skimage import exposure
from skimage.filters import gaussian, roberts, sobel, sobel_h

im = io.imread('S1B_EW_GRDM_1SDH_20180206T050415_Botnicka.tif')
aoi = im[4600:5200,3800:4400]
ice = exposure.equalize_hist(aoi)

# Filtracja wstępna
# Usunięcie szumów
G = gaussian(ice, sigma=1.5, preserve_range=True) 
# Dodatkowo wydobycie granic
R = roberts(G)                                       

In [21]:
from skimage.segmentation import felzenszwalb, mark_boundaries

# Segmentacja orginalnego pliku
O_fz = felzenszwalb(ice, scale=100, sigma=0.5, min_size=200)
# Segmentacja pliku po usunięciu szumów
G_fz = felzenszwalb(G, scale=100, sigma=0.5, min_size=200)
# Segmentacja pliku po usunięciu szumów i podkreśleniu granic
R_fz = felzenszwalb(R, scale=100, sigma=0.5, min_size=200)

fig = plt.figure(figsize=(30,30))
ax0 = plt.subplot(221)
ax0.imshow(mark_boundaries(ice, O_fz))
ax0.set_title("Felzenszwalb segmentation of the input image",fontsize=25)
ax3 = plt.subplot(222)
ax3.imshow(mark_boundaries(exposure.equalize_hist(G), G_fz))
ax3.set_title("Felzenszwalb segmentation of image after gassian filtration",fontsize=25)
ax6 = plt.subplot(223)
ax6.imshow(mark_boundaries(ice, R_fz))
ax6.set_title("Felzenszwalb segmentationof image after gassian + roberts filtration",fontsize=25)
ax6 = plt.subplot(224)
# pokazanie jak wygląda raster wynikowy - osobne ID dla każdego obiektu
ax6.imshow(ice,cmap=plt.get_cmap('gray'))
ax6.imshow(R_fz,cmap=plt.get_cmap('tab20'),alpha=.7)
ax6.set_title("Felzenszwalb segmentation",fontsize=25)


plt.show()

In [22]:
# Porównanie ilości obiektów wynikowych
fig = plt.figure(figsize=(30,30))
ax0 = plt.subplot(131)
ax0.imshow(O_fz,cmap=plt.get_cmap('tab20'))
ax0.set_title("Segmented input image: %d segments" % (len(np.unique(O_fz))),fontsize=25)
ax3 = plt.subplot(132)
ax3.imshow((G_fz),cmap=plt.get_cmap('tab20'))
ax3.set_title("Segmented after gaussian filtration: %d segments" % (len(np.unique(G_fz))),fontsize=25)
ax6 = plt.subplot(133)
ax6.imshow(R_fz,cmap=plt.get_cmap('tab20'))
ax6.set_title("Segmented after gaussian + roberts filtration: %d segments" % (len(np.unique(R_fz))),fontsize=25)

plt.tight_layout()
plt.show()

In [23]:
from matplotlib import colors
from skimage.segmentation import find_boundaries

# Kontury w postaci maskę rastrowej typu True/False

lines_O_fz = find_boundaries(O_fz,mode='inner',background=0)
lines_G_fz = find_boundaries(G_fz,mode='inner',background=0)
lines_R_fz = find_boundaries(R_fz,mode='inner',background=0)

cm = colors.ListedColormap(["#FFFFFF00", '#FFFF00'])

fig = plt.figure(figsize=(30,30))
ax0 = plt.subplot(131)
ax0.imshow(ice*0,cmap=plt.get_cmap('gray'))
ax0.imshow(lines_O_fz,cmap=cm)
ax0.set_title("Segmentation of the input image",fontsize=20)
ax1 = plt.subplot(132)
ax1.imshow(ice*0,cmap=plt.get_cmap('gray'))
ax1.imshow(lines_G_fz,cmap=cm)
ax1.set_title("Segmentation of image after gassian filtration",fontsize=20)
ax2 = plt.subplot(133)
ax2.imshow(ice*0,cmap=plt.get_cmap('gray'))
ax2.imshow(lines_R_fz,cmap=cm)
ax2.set_title("Segmentationof image after gassian + roberts filtration",fontsize=20)

plt.tight_layout()
plt.show()

In [24]:
from skimage import measure

# Kontury jako linie (wektorowo)

contours = measure.find_contours(G, 0.37)

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(G,cmap=plt.get_cmap('gray'))

for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2, color='yellow')

plt.show()
print(type(contours))

<i>-------Charakterystyka obiektów-------</i>

In [25]:
from skimage.measure import label, regionprops

regions = G_fz
label_img = label(regions)
all_properties = regionprops(label_img, intensity_image=ice)

for region in all_properties:
    print("ID: %3d, area: %5d, eccentricity: %3.3f, solidity: %3.3f, mean intensity: %3.3f" 
          % (region.label,region.area,region.eccentricity,region.solidity,region.mean_intensity))

In [26]:
from skimage import img_as_bool, io, color, morphology

# Tworzenie linii szkieletowych obiektów - sprawdzanie rozgałęzienia
skeleton = morphology.medial_axis(R_fz)

cm = colors.ListedColormap(["#FFFFFF00", '#FFFF00'])

fig = plt.figure(figsize=(10,10))
ax0 = plt.subplot()
ax0.imshow(R_fz,cmap=plt.get_cmap('gray'))
ax0.imshow(skeleton,cmap=cm)
plt.show()

<i>-------Prosta klasyfikacja obiektów-------</i>

In [27]:
#area, mean_intensity
criterion = [region.mean_intensity for region in all_properties]
plt.hist(criterion,bins=30)
plt.show()

In [29]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3)
criterion = np.array(criterion).reshape(-1,1)
model.fit(criterion)
#print(model.cluster_centers_)

region_labels = model.predict(criterion)

classes = label_img.copy()
for class_id,region in zip(region_labels, all_properties):
    classes[tuple(region.coords.T)] = class_id

fig = plt.figure(figsize=(10,10))
ax0 = plt.subplot(121)
ax0.imshow(ice,cmap=plt.get_cmap('gray'))
ax0.imshow(classes,cmap=plt.get_cmap('tab20'),alpha=.7)
ax0.set_title("Classified image",fontsize=15)
ax3 = plt.subplot(122)
ax3.imshow(ice,cmap=plt.get_cmap('gray'))
ax3.set_title("Starting image",fontsize=15)

plt.show()

<i>-------Detekcja jasnych obiektów na przykładzie chmur-------</i>

In [17]:
# Wczytanie obrazu zarejestrowanego o 25.09.2018 9:50:29 przez MSI, Sentinel-2, ESA
from skimage import exposure, transform

gmg = io.imread('S2B_MSIL1C_20180925T095029_N0206_R079_clouds.tif', as_grey=True)

foto = img_as_ubyte(contrast_perc(gmg, 1, 98))
# subsampling
n = 6
clouds = transform.downscale_local_mean(foto,(n,n))

C:\ProgramData\Anaconda3\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')
C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [30]:
from skimage import measure

# Znajdż kontury i zapisz jako linię (wektorowo)

contours = measure.find_contours(clouds, 40)

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(clouds,cmap=plt.get_cmap('gray'))

for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=1, color='cyan')

plt.show()
print(type(contours))

In [31]:
from skimage.feature import blob_dog, blob_log, blob_doh
import math

def radius(sigma):
    return(math.sqrt(2) * sigma)

# blobs_log[row,column,sigma], gdzie promień = sqrt(2) * sigma
blobs_log = blob_log(foto, min_sigma=1, max_sigma=100)
blobs_doh = blob_doh(foto, min_sigma=0.02, max_sigma=50)
blobs_dog = blob_dog(foto, min_sigma=1, max_sigma=100, threshold=.01)

# oblicz promień
for blobtype in (blobs_log,blobs_doh,blobs_dog):
    if blobtype.shape[0] > 0:
        blobtype[:,2] = radius(blobtype[:,2])

fig = plt.figure(figsize=(30,30))

ax3 = plt.subplot(221)
ax3.imshow(foto,cmap=plt.get_cmap('gray'))
ax3.set_title("Grayscale image",fontsize=20)

ax0 = plt.subplot(222)
ax0.imshow(foto,cmap=plt.get_cmap('gray'))
ax0.set_title('Laplacian of Gaussian',fontsize=20)
for b in blobs_log:
    O = plt.Circle((b[1], b[0]), b[2], color='lightgreen', fill=False)#, linewidth=2) 
    ax0.add_patch(O)

ax1 = plt.subplot(223)
ax1.imshow(foto,cmap=plt.get_cmap('gray'))
ax1.set_title('Determinant of Hessian',fontsize=20)
for b in blobs_doh:
    O = plt.Circle((b[1], b[0]), b[2], color='red', fill=False)#, linewidth=2) 
    ax1.add_patch(O)

ax2 = plt.subplot(224)
ax2.imshow(foto,cmap=plt.get_cmap('gray'))
ax2.set_title('Difference of Gaussian',fontsize=20)
for b in blobs_dog:
    O = plt.Circle((b[1], b[0]), b[2], color='yellow', fill=False)#, linewidth=2) 
    ax2.add_patch(O)


plt.show()